In [2]:

%pip install evaluate
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Print GPU status
if torch.cuda.is_available():
    print(f"✓ GPU is available and will be used: {torch.cuda.get_device_name(0)}")
    print(f"  Number of GPUs: {torch.cuda.device_count()}")
else:
    print("✗ No GPU detected. Training will use CPU.")
    
print(f"Device: {device}")

imdb = load_dataset("imdb")

train_dataset = imdb["train"].shuffle(seed=42).select([i for i in list(range(10000))])
test_dataset = imdb["test"].shuffle(seed=42).select([i for i in list(range(10000))])

checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def preprocess_function(examples):
   return tokenizer(examples["text"], truncation=True, padding=True, max_length=132)

tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

def compute_metrics(eval_pred):
   load_accuracy = evaluate.load("accuracy")
   load_f1 = evaluate.load("f1")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=0.0001,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    use_cpu=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.4 MB/s eta 0:00:00
✓ GPU is available and will be used: Tesla T4
  Number of GPUs: 1
Device: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

...

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

There were missing keys in the checkpoint model loaded: ['distilbert.embeddings.LayerNorm.weight', 'distilbert.embeddings.LayerNorm.bias'].
There were unexpected keys in the checkpoint model loaded: ['distilbert.embeddings.LayerNorm.beta', 'distilbert.embeddings.LayerNorm.gamma'].


TrainOutput(global_step=939, training_loss=0.2272502220730685, metrics={'train_runtime': 528.0641, 'train_samples_per_second': 56.811, 'train_steps_per_second': 1.778, 'total_flos': 1024552536480000.0, 'train_loss': 0.2272502220730685, 'epoch': 3.0})

In [5]:
from transformers import TextClassificationPipeline
sentiment = TextClassificationPipeline(model=model, tokenizer=tokenizer)
sentiment([
    "Absolutely terrible waste of time",
    "The acting was phenomenal and emotionally powerful",
    "It was okay, nothing special",
    "Worst film of the decade",
    "I would definitely recommend this to my friends",
    "I would not recommend this to my worst enemy"
])


[{'label': 'LABEL_0', 'score': 0.9868901968002319},
 {'label': 'LABEL_1', 'score': 0.9883437156677246},
 {'label': 'LABEL_1', 'score': 0.6392606496810913},
 {'label': 'LABEL_0', 'score': 0.9725374579429626},
 {'label': 'LABEL_1', 'score': 0.9877927899360657},
 {'label': 'LABEL_0', 'score': 0.8129397034645081}]